In [3]:
from os import sep
from re import LOCALE
import re
import dash
from dash import dependencies
import dash_core_components as dcc
import dash_html_components as html
from pandas.core.accessor import PandasDelegate
import plotly.express as px
import pandas as pd
from datetime import datetime

In [42]:
df = pd.read_csv('output/bases/210726-1027-210713_panel_nc.csv', sep=';', dtype='object')

df['F_COMPRA'] = pd.to_datetime(df['F_COMPRA'])
df['F_COMPRA_ANT'] = pd.to_datetime(df['F_COMPRA_ANT'])

df.loc[:,'CANT'] = -pd.to_numeric(df['CANT'])
df.loc[:,'TOTAL_COSTO'] = pd.to_numeric(df['TOTAL_COSTO'])

#df['NLOCAL_CREACION'] = df['NLOCAL_CREACION'].astype('category')

df['MES_NC'] = df['F_COMPRA'].dt.month
df['MES_COMPRA'] = df['F_COMPRA_ANT'].dt.month

#df['MES_NC'] = df['F_COMPRA'].dt.strftime('%b')
#df['MES_COMPRA'] = df['F_COMPRA_ANT'].dt.strftime('%b')

#dftiendas1 = df[(df['NLOCAL_CREACION']!='FALABELLA. COM ALTERNO')&(df['NLOCAL_CREACION']!='ADMINISTRATIVO')&(df['NLOCAL_CREACION']!='VENTA EMPRESA')]
#dftiendas = dftiendas1.groupby(['NLOCAL_CREACION','MES_NC']).agg({'CANT':'sum','TOTAL_COSTO':'sum','CAUTORIZA':'nunique'}).reset_index()


In [5]:
df.head()

,COD_TRX,CAUTORIZA,F_COMPRA,LOCAL_CREACION,NLOCAL_CREACION,NRO_TERM_NVA,SEC_NVA,CVEND_NVO,LOCAL_ANT,NLOCAL_ANT,...,DESC_SKU,CANT,MNETO,TOTAL_COSTO,CTIPIFICACION,XTIPIFICACION,MES_NC_NUM,MES_COMPRA_NUM,MES_NC,MES_COMPRA
0,205246708,1059705755,2021-01-02,19,BUENAVISTA,26,3171,924121,2000,TIENDA FALABELLA .COM,...,REEBOK RUNNER 4.0/VE0/9/40,1,-151252,90895.305,D1,Devolucion de producto,1,12,Jan,Dec
1,205245636,1059717291,2021-01-02,19,BUENAVISTA,19,3342,103500,19,BUENAVISTA,...,BERMUDA JUV DEN BED CORDEL RAW 32,1,-33605,28472.094,D1,Devolucion de producto,1,12,Jan,Dec
2,205255000,1059725210,2021-01-02,56,LA CAROLA,8,8605,959618,2000,TIENDA FALABELLA .COM,...,EC-PIJAMA FLEECE ANTIDESL NINA 24M,1,-22597,23471.000,D1,Devolucion de producto,1,11,Jan,Nov
3,205259670,1059758904,2021-01-02,56,LA CAROLA,18,3687,937312,56,LA CAROLA,...,SHORT ATP BEARCLIFF NAVY L,1,-31024,15164.000,D1,Devolucion de producto,1,12,Jan,Dec
4,205259670,1059758904,2021-01-02,56,LA CAROLA,18,3687,937312,56,LA CAROLA,...,SHORT PIJAMA PCJ BTP2 B COMBO B L,1,-23266,12016.808,D1,Devolucion de producto,1,12,Jan,Dec


In [6]:
df.describe()

,CANT,TOTAL_COSTO,MES_NC_NUM,MES_COMPRA_NUM
count,378701.000000,3.787010e+05,378701.000000,378701.00000
mean,1.290504,1.356247e+05,3.315133,4.75177
std,24.784683,7.931431e+05,1.812502,3.45584
min,-22.000000,-2.263512e+06,1.000000,1.00000
25%,1.000000,1.899104e+04,2.000000,2.00000
50%,1.000000,3.059092e+04,3.000000,4.00000
75%,1.000000,7.141500e+04,5.000000,6.00000
max,7338.000000,1.730241e+08,6.000000,12.00000


In [8]:
df.describe(include=['category'])

,NLOCAL_CREACION
count,378701
unique,36
top,FALABELLA. COM ALTERNO
freq,54529


In [12]:
aux = pd.concat([df.head(20), df.tail(20)])

In [13]:
aux.groupby(['LOCAL_CREACION', 'MES_NC_NUM'])['CAUTORIZA'].transform('nunique')

0          2
1          2
2          2
3          2
4          2
5          2
6          2
7          2
8          2
9         10
10        10
11        10
12        10
13        10
14        10
15        10
16        10
17        10
18        10
19        10
378681     4
378682     4
378683     4
378684     4
378685     1
378686     1
378687     1
378688     1
378689     1
378690     1
378691     2
378692     1
378693     2
378694     1
378695     1
378696     1
378697     2
378698     2
378699     1
378700     1
Name: CAUTORIZA, dtype: int64

In [15]:
aux.groupby(['LOCAL_CREACION', 'MES_NC_NUM'])['CAUTORIZA'].nunique()

LOCAL_CREACION  MES_NC_NUM
108             6              1
123             6              2
13              6              1
138             1             10
                6              1
183             6              2
19              1              2
25              6              1
3000            6              4
35              6              1
43              6              1
45              6              1
56              1              2
72              1              2
                6              1
82              6              1
98              6              1
Name: CAUTORIZA, dtype: int64

In [30]:
df['NC_X_LOCAL_MES'] = df.groupby(['LOCAL_CREACION', 'CVEND_NVO', 'MES_NC'])['CAUTORIZA'].transform('nunique')

In [68]:
df['NC_MEAN_LOCAL_MES'] = df.groupby(['LOCAL_CREACION', 'MES_NC'])['CAUTORIZA'].transform('nunique')

In [17]:
lmc = df.groupby(['LOCAL_CREACION', 'MES_NC_NUM', 'CVEND_NVO'])['CAUTORIZA'].nunique().reset_index()

In [69]:
lmc.groupby(['LOCAL_CREACION', 'MES_NC']).mean()

CAUTORIZA
LOCAL_CREACION MES_NC           
101            Apr     11.557143
               Feb     13.750000
               Jan     14.587500
               Jun     15.464789
               Mar     12.547945
...                          ...
99             Feb      3.000000
               Jan      8.500000
               Jun      3.000000
               Mar      6.500000
               May      1.500000

[206 rows x 1 columns]

In [31]:
lmc.groupby(['LOCAL_CREACION', 'MES_NC']).mean()

KeyError: 'MES_NC'

In [43]:
mean_mes = df.groupby(['NLOCAL_CREACION', 'MES_NC', 'CVEND_NVO'])['CAUTORIZA'].nunique().reset_index().groupby(['NLOCAL_CREACION', 'MES_NC']).mean().reset_index()

In [44]:
fig = px.line(mean_mes, x='MES_NC', y='CAUTORIZA', color='NLOCAL_CREACION')
fig.show()

In [45]:
mean_mes

,NLOCAL_CREACION,MES_NC,CAUTORIZA
0,ACQUA,1,32.929577
1,ACQUA,2,29.030303
2,ACQUA,3,22.426230
3,ACQUA,4,13.033898
4,ACQUA,5,13.947368
...,...,...,...
201,WTC CALI,2,27.184783
202,WTC CALI,3,28.433735
203,WTC CALI,4,16.507042
204,WTC CALI,5,18.085714


In [75]:
a = df.groupby(['NLOCAL_CREACION', 'MES_NC', 'CVEND_NVO'])['CAUTORIZA'].nunique().reset_index()
a['MEAN_TIENDA_MES'] = a.groupby(['NLOCAL_CREACION','MES_NC'])['CAUTORIZA'].transform('mean')


In [83]:
a[a['CAUTORIZA']>a['MEAN_TIENDA_MES']*2]

,NLOCAL_CREACION,MES_NC,CVEND_NVO,CAUTORIZA,MEAN_TIENDA_MES
20,ACQUA,1,155705,68,32.929577
26,ACQUA,1,162507,79,32.929577
27,ACQUA,1,168688,102,32.929577
44,ACQUA,1,910538,168,32.929577
45,ACQUA,1,910562,124,32.929577
...,...,...,...,...,...
12403,WTC CALI,6,897361,168,29.894118
12404,WTC CALI,6,904414,298,29.894118
12407,WTC CALI,6,918458,234,29.894118
12409,WTC CALI,6,923567,103,29.894118


In [101]:
a[(a['NLOCAL_CREACION']=='ACQUA')&(a['CAUTORIZA']>a['MEAN_TIENDA_MES']*2)].groupby('MES_NC')['CAUTORIZA'].nlargest(3)

MES_NC     
1       44     168
        69     151
        62     146
2       114    161
        126    118
        127    112
3       175    108
        172     91
        187     88
4       221     54
        216     52
        236     49
5       280     62
        292     45
        277     41
6       332    106
        348     79
        358     71
Name: CAUTORIZA, dtype: int64

In [109]:
a[(a['NLOCAL_CREACION']=='ACQUA')&(a['CAUTORIZA']>a['MEAN_TIENDA_MES']*2)]

,NLOCAL_CREACION,MES_NC,CVEND_NVO,CAUTORIZA,MEAN_TIENDA_MES
20,ACQUA,1,155705,68,32.929577
26,ACQUA,1,162507,79,32.929577
27,ACQUA,1,168688,102,32.929577
44,ACQUA,1,910538,168,32.929577
45,ACQUA,1,910562,124,32.929577
...,...,...,...,...,...
360,ACQUA,6,974234,46,20.285714
361,ACQUA,6,974242,47,20.285714
363,ACQUA,6,977373,57,20.285714
365,ACQUA,6,981850,50,20.285714


In [108]:
a[(a['NLOCAL_CREACION']=='UNICENTRO')&(a['MES_NC']==2)].describe()

,MES_NC,CAUTORIZA,MEAN_TIENDA_MES
count,113.0,113.000000,113.000000
mean,2.0,19.752212,19.752212
std,0.0,50.855455,0.000000
min,2.0,1.000000,19.752212
25%,2.0,2.000000,19.752212
50%,2.0,5.000000,19.752212
75%,2.0,13.000000,19.752212
max,2.0,473.000000,19.752212


In [88]:
a.NLOCAL_CREACION.unique()

array(['ACQUA', 'ADMINISTRATIVO', 'ARKADIA', 'BUENAVISTA',
       'C&B Online ADMINISTRATIVO', 'CACIQUE', 'CARACOLI', 'CENTRO MAYOR',
       'COLINA', 'Crate and Barrel Colina', 'Crate and Barrel Online',
       'DIVER PLAZA', 'EL CASTILLO', 'FALABELLA. COM ALTERNO', 'FONTANAR',
       'GALERIAS', 'HAYUELOS', 'JARDIN PLAZA', 'LA CAROLA',
       'LA FELICIDAD', 'MARTINA COLINA', 'MARTINA EL TESORO',
       'MARTINA FONTANAR', 'PARQUE ARBOLEDA', 'PLAZA CENTRAL',
       'PRIMAVERA', 'SAN DIEGO', 'SANTA FE', 'SANTAFE MEDELLIN', 'SUBA',
       'TIENDA FALABELLA .COM', 'TITAN', 'UNICENTRO', 'VENTA EMPRESA',
       'VILLA COUNTRY', 'WTC CALI'], dtype=object)

In [104]:
a[(a['NLOCAL_CREACION']=='ACQUA')&(a['CAUTORIZA']>a['MEAN_TIENDA_MES']*2)].groupby('MES_NC')['CAUTORIZA'].nlargest(2)

MES_NC     
1       44     168
        69     151
2       114    161
        126    118
3       175    108
        172     91
4       221     54
        216     52
5       280     62
        292     45
6       332    106
        348     79
Name: CAUTORIZA, dtype: int64